# G02: Phase Lag Index (PLI)

**Duration**: 55 minutes  
**Prerequisites**: G01 (Phase Locking Value), C01 (Volume Conduction Problem)  
**Next**: G03 (Weighted Phase Lag Index)

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Explain why PLV fails for volume conduction
2. Define PLI as asymmetry of phase difference distribution
3. Implement PLI computation
4. Interpret PLI values (0 to 1)
5. Understand the sign inconsistency issue with PLI
6. Apply PLI to hyperscanning analysis
7. Recognize when to use PLI vs PLV

In [ ]:
# Required imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from typing import Tuple, Any

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Color palette
COLORS = {
    'signal_1': '#2E86AB',
    'signal_2': '#E94F37',
    'subject_1': '#2E86AB',
    'subject_2': '#A23B72',
    'highlight': '#F18F01',
    'warning': '#C73E1D'
}

---

## Section 1: The Problem with PLV (Revisited)

In G01, we learned that PLV measures phase consistency—but we also saw its critical flaw: **volume conduction**.

When a single electrical source in the brain spreads through conductive tissue to multiple electrodes, those electrodes record essentially the same signal. This means:
- Phase difference ≈ 0 (or π)
- Phase difference is extremely consistent (always near 0)
- PLV = very high!

But this is **spurious**—not true connectivity, just physics!

**What we need**: A metric that **ignores zero-phase relationships**.

> **Key message**: "PLI ignores phase differences at 0 and π, eliminating volume conduction artifacts."

In [ ]:
# Visualization 1: Phase difference distributions

np.random.seed(42)
n_samples = 10000

# Three scenarios
# 1. Volume conduction: peaked at 0
phase_diff_vc = np.random.normal(0, 0.15, n_samples)

# 2. True connection: peaked elsewhere (e.g., 45°)
phase_diff_true = np.random.normal(np.pi/4, 0.3, n_samples)

# 3. No connection: uniform
phase_diff_none = np.random.uniform(-np.pi, np.pi, n_samples)

# Compute PLV for each
def quick_plv(phase_diff):
    return np.abs(np.mean(np.exp(1j * phase_diff)))

plv_vc = quick_plv(phase_diff_vc)
plv_true = quick_plv(phase_diff_true)
plv_none = quick_plv(phase_diff_none)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

scenarios = [
    (phase_diff_vc, plv_vc, 'Volume Conduction', COLORS['warning']),
    (phase_diff_true, plv_true, 'True Connection', COLORS['signal_1']),
    (phase_diff_none, plv_none, 'No Connection', 'gray')
]

for ax, (pd, plv, title, color) in zip(axes, scenarios):
    ax.hist(pd, bins=50, color=color, alpha=0.7, density=True, range=(-np.pi, np.pi))
    ax.axvline(x=0, color='black', linestyle='--', linewidth=2, label='Zero lag')
    ax.set_xlabel('Phase Difference (rad)')
    ax.set_ylabel('Density')
    ax.set_title(f'{title}\nPLV = {plv:.3f}', fontweight='bold')
    ax.set_xlim(-np.pi, np.pi)
    ax.legend()

plt.suptitle("PLV Can't Distinguish Volume Conduction from True Connectivity\n(Both left scenarios have HIGH PLV!)", 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 2: The Key Insight — Phase Difference Sign

The brilliant insight behind PLI is to look at the **sign** of sin(Δφ):

- **sin(Δφ) > 0** when 0 < Δφ < π → Y leads X
- **sin(Δφ) < 0** when -π < Δφ < 0 → X leads Y  
- **sin(Δφ) = 0** when Δφ = 0 or π → **Volume conduction zone!**

**PLI principle**: 
- True connection with consistent lag → signs mostly same → **asymmetric**
- Volume conduction (Δφ ≈ 0) → noise makes signs flip equally → **symmetric**

PLI measures this **asymmetry of the sign distribution**.

In [ ]:
# Visualization 2: Sign of sin(Δφ) on unit circle

fig, ax = plt.subplots(figsize=(8, 8))

# Draw unit circle
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=2)

# Fill upper half (positive sin)
theta_upper = np.linspace(0, np.pi, 50)
ax.fill_between(np.cos(theta_upper), 0, np.sin(theta_upper), 
                color=COLORS['signal_1'], alpha=0.3, label='sin(Δφ) > 0: Y leads X')

# Fill lower half (negative sin)
theta_lower = np.linspace(np.pi, 2*np.pi, 50)
ax.fill_between(np.cos(theta_lower), 0, np.sin(theta_lower), 
                color=COLORS['signal_2'], alpha=0.3, label='sin(Δφ) < 0: X leads Y')

# Mark volume conduction zone
ax.scatter([1, -1], [0, 0], s=200, c=COLORS['warning'], zorder=5, 
           edgecolor='black', linewidth=2, label='sin(Δφ) = 0: Volume conduction zone')

# Axes
ax.axhline(y=0, color='black', linewidth=1)
ax.axvline(x=0, color='gray', linewidth=1, alpha=0.5)

# Labels
ax.annotate('Δφ = 0', xy=(1.1, 0.1), fontsize=12)
ax.annotate('Δφ = π', xy=(-1.3, 0.1), fontsize=12)
ax.annotate('Δφ = π/2', xy=(0.1, 1.1), fontsize=12)
ax.annotate('Δφ = -π/2', xy=(0.1, -1.15), fontsize=12)

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_xlabel('cos(Δφ)', fontsize=12)
ax.set_ylabel('sin(Δφ)', fontsize=12)
ax.set_title('Phase Difference Sign: Positive, Negative, or Zero', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: Sign balance vs imbalance

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

np.random.seed(42)
n_points = 200

# Volume conduction: phase diffs fluctuate around 0
phase_diff_vc = np.random.normal(0, 0.3, n_points)
signs_vc = np.sign(np.sin(phase_diff_vc))

# True connection: consistent positive lag
phase_diff_true = np.random.normal(np.pi/3, 0.3, n_points)
signs_true = np.sign(np.sin(phase_diff_true))

# Plot volume conduction
ax = axes[0]
colors_vc = [COLORS['signal_1'] if s > 0 else COLORS['signal_2'] for s in signs_vc]
ax.scatter(range(len(signs_vc)), signs_vc, c=colors_vc, alpha=0.6, s=30)
ax.axhline(y=0, color='gray', linestyle='--')
ax.set_xlabel('Time point')
ax.set_ylabel('Sign of sin(Δφ)')
ax.set_ylim(-1.5, 1.5)
ax.set_yticks([-1, 0, 1])
pos_count = np.sum(signs_vc > 0)
neg_count = np.sum(signs_vc < 0)
ax.set_title(f'Volume Conduction\n+1: {pos_count}, -1: {neg_count} (BALANCED)', fontweight='bold')

# Plot true connection
ax = axes[1]
colors_true = [COLORS['signal_1'] if s > 0 else COLORS['signal_2'] for s in signs_true]
ax.scatter(range(len(signs_true)), signs_true, c=colors_true, alpha=0.6, s=30)
ax.axhline(y=0, color='gray', linestyle='--')
ax.set_xlabel('Time point')
ax.set_ylabel('Sign of sin(Δφ)')
ax.set_ylim(-1.5, 1.5)
ax.set_yticks([-1, 0, 1])
pos_count = np.sum(signs_true > 0)
neg_count = np.sum(signs_true < 0)
ax.set_title(f'True Connection\n+1: {pos_count}, -1: {neg_count} (IMBALANCED)', fontweight='bold')

plt.suptitle('Sign Asymmetry Distinguishes True Connectivity', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 3: The PLI Formula

The **Phase Lag Index** (Stam et al., 2007) is defined as:

$$PLI = \left| \frac{1}{N} \sum_{t=1}^{N} sign(\sin(\Delta\phi(t))) \right| = \left| \langle sign(\sin(\Delta\phi)) \rangle \right|$$

**Interpretation**:
1. Compute sin(Δφ) at each time point
2. Take the sign: +1, -1, or 0
3. Average the signs
4. Take absolute value

**Properties**:
- Range: 0 to 1
- **PLI = 0**: Symmetric (could be volume conduction OR no connection)
- **PLI = 1**: Perfect asymmetry (all phases on same side)
- PLI ignores **magnitude** of phase difference, only sign matters

In [ ]:
# Visualization 4: PLI computation steps

np.random.seed(42)
n_samples = 500
t = np.arange(n_samples)

# Generate phase differences for true connection
phase_diff = np.random.normal(np.pi/4, 0.4, n_samples)

fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)

# Step 1: Phase difference
axes[0].plot(t, phase_diff, color=COLORS['signal_1'], alpha=0.7)
axes[0].axhline(y=0, color='gray', linestyle='--')
axes[0].set_ylabel('Δφ (rad)')
axes[0].set_title('Step 1: Phase Difference Δφ(t)', fontweight='bold')

# Step 2: sin(Δφ)
sin_phase_diff = np.sin(phase_diff)
axes[1].plot(t, sin_phase_diff, color=COLORS['signal_2'], alpha=0.7)
axes[1].axhline(y=0, color='gray', linestyle='--')
axes[1].set_ylabel('sin(Δφ)')
axes[1].set_title('Step 2: Compute sin(Δφ)', fontweight='bold')

# Step 3: sign(sin(Δφ))
signs = np.sign(sin_phase_diff)
colors = [COLORS['signal_1'] if s > 0 else COLORS['signal_2'] if s < 0 else 'gray' for s in signs]
axes[2].scatter(t, signs, c=colors, alpha=0.5, s=10)
axes[2].axhline(y=0, color='gray', linestyle='--')
axes[2].set_ylabel('sign(sin(Δφ))')
axes[2].set_yticks([-1, 0, 1])
axes[2].set_title('Step 3: Extract Sign (+1, -1, or 0)', fontweight='bold')

# Step 4: PLI
mean_sign = np.mean(signs)
pli = np.abs(mean_sign)

axes[3].bar(['Mean of signs', 'PLI = |mean|'], [mean_sign, pli], 
            color=[COLORS['signal_1'], COLORS['highlight']], edgecolor='black')
axes[3].axhline(y=0, color='gray', linestyle='-')
axes[3].set_ylim(-1, 1)
axes[3].set_ylabel('Value')
axes[3].set_title(f'Step 4: Average Signs and Take Absolute Value → PLI = {pli:.3f}', fontweight='bold')

plt.xlabel('Time point')
plt.suptitle('PLI Computation Steps', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 4: Implementing PLI

The pipeline is very similar to PLV:

1. Band-pass filter both signals
2. Extract instantaneous phases (Hilbert)
3. Compute phase difference Δφ(t)
4. Compute **sign(sin(Δφ(t)))** ← key difference!
5. Average the signs
6. Take absolute value

In [ ]:
# Function 1: compute_pli

def compute_pli(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    filter_order: int = 4
) -> float:
    """
    Compute Phase Lag Index between two signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal (1D array).
    y : NDArray[np.float64]
        Second signal (1D array).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band of interest (low, high) in Hz.
    filter_order : int, optional
        Order of the Butterworth filter (default: 4).
    
    Returns
    -------
    float
        Phase Lag Index between 0 and 1.
    
    Notes
    -----
    PLI = |mean(sign(sin(phase_x - phase_y)))|
    
    PLI is robust to volume conduction because zero-lag
    relationships (Δφ ≈ 0) contribute equally to +1 and -1.
    """
    # Design bandpass filter
    nyq = fs / 2
    low, high = band[0] / nyq, band[1] / nyq
    b, a = signal.butter(filter_order, [low, high], btype='band')
    
    # Filter signals
    x_filt = signal.filtfilt(b, a, x)
    y_filt = signal.filtfilt(b, a, y)
    
    # Extract phases via Hilbert transform
    phase_x = np.angle(signal.hilbert(x_filt))
    phase_y = np.angle(signal.hilbert(y_filt))
    
    # Compute PLI
    phase_diff = phase_x - phase_y
    pli = np.abs(np.mean(np.sign(np.sin(phase_diff))))
    
    return float(pli)

In [ ]:
# Function 2: compute_pli_from_phases

def compute_pli_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute PLI from pre-computed phases.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of first signal (radians).
    phase_y : NDArray[np.float64]
        Instantaneous phase of second signal (radians).
    
    Returns
    -------
    float
        Phase Lag Index between 0 and 1.
    """
    phase_diff = phase_x - phase_y
    pli = np.abs(np.mean(np.sign(np.sin(phase_diff))))
    return float(pli)

In [ ]:
# Function 3: compute_sign_series

def compute_sign_series(
    phase_diff: NDArray[np.float64]
) -> NDArray[np.int64]:
    """
    Compute sign of sin(phase_diff) time series.
    
    Parameters
    ----------
    phase_diff : NDArray[np.float64]
        Phase difference time series (radians).
    
    Returns
    -------
    NDArray[np.int64]
        Sign series: +1 (Y leads), -1 (X leads), or 0 (zero lag).
    """
    return np.sign(np.sin(phase_diff)).astype(np.int64)

In [ ]:
# Visualization 5: PLI vs PLV pipeline comparison

fig, axes = plt.subplots(2, 4, figsize=(16, 6))

# Common steps
steps_common = ['1. Filter', '2. Hilbert', '3. Phases', '4. Δφ']
steps_plv = ['5. e^(iΔφ)', '6. Average', '7. |·|', 'PLV']
steps_pli = ['5. sin(Δφ)', '6. sign(·)', '7. Average', '8. |·| → PLI']

# PLV path (top row)
for i, step in enumerate(steps_common[:4]):
    axes[0, i].text(0.5, 0.5, step, ha='center', va='center', fontsize=12, fontweight='bold')
    axes[0, i].set_xlim(0, 1)
    axes[0, i].set_ylim(0, 1)
    axes[0, i].axis('off')
    if i < 3:
        axes[0, i].annotate('', xy=(1.1, 0.5), xytext=(0.9, 0.5),
                           arrowprops=dict(arrowstyle='->', color='gray'))

# PLI path (bottom row) - diverges at step 5
for i in range(4):
    if i < 3:
        axes[1, i].text(0.5, 0.5, '↓', ha='center', va='center', fontsize=20, color='gray')
    else:
        axes[1, i].text(0.5, 0.7, 'PLV path:', ha='center', va='center', fontsize=10, color=COLORS['signal_1'])
        axes[1, i].text(0.5, 0.5, 'e^(iΔφ) → avg → |·|', ha='center', va='center', fontsize=10, color=COLORS['signal_1'])
        axes[1, i].text(0.5, 0.3, 'PLI path:', ha='center', va='center', fontsize=10, color=COLORS['signal_2'])
        axes[1, i].text(0.5, 0.1, 'sin → sign → avg → |·|', ha='center', va='center', fontsize=10, color=COLORS['signal_2'])
    axes[1, i].set_xlim(0, 1)
    axes[1, i].set_ylim(0, 1)
    axes[1, i].axis('off')

plt.suptitle('PLI vs PLV: Same Start, Different Finish', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 5: PLI vs Volume Conduction

Now let's demonstrate PLI's key advantage: **robustness to volume conduction**.

In [ ]:
# Function 4: demonstrate_pli_volume_conduction

def demonstrate_pli_volume_conduction(
    n_samples: int = 10000,
    fs: float = 500.0,
    frequency: float = 10.0,
    seed: int | None = None
) -> dict[str, Any]:
    """
    Compare PLV and PLI for volume conduction vs true connectivity.
    
    Parameters
    ----------
    n_samples : int
        Number of samples.
    fs : float
        Sampling frequency in Hz.
    frequency : float
        Oscillation frequency in Hz.
    seed : int | None
        Random seed for reproducibility.
    
    Returns
    -------
    dict[str, Any]
        PLV and PLI values for both scenarios.
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(n_samples) / fs
    band = (frequency - 2, frequency + 2)
    
    # Volume conduction: same signal + small noise
    source = np.sin(2 * np.pi * frequency * t)
    noise_level = 0.1
    vc_1 = source + noise_level * np.random.randn(n_samples)
    vc_2 = source + noise_level * np.random.randn(n_samples)
    
    # True connectivity: signal with consistent delay
    delay_samples = int(0.015 * fs)  # 15ms delay (~54° at 10 Hz)
    tc_1 = source + 0.3 * np.random.randn(n_samples)
    tc_2 = np.roll(source, delay_samples) + 0.3 * np.random.randn(n_samples)
    
    # Compute metrics
    # PLV helper
    def compute_plv(x, y, fs, band):
        nyq = fs / 2
        b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
        x_filt = signal.filtfilt(b, a, x)
        y_filt = signal.filtfilt(b, a, y)
        phase_x = np.angle(signal.hilbert(x_filt))
        phase_y = np.angle(signal.hilbert(y_filt))
        return np.abs(np.mean(np.exp(1j * (phase_x - phase_y))))
    
    return {
        'vc_plv': float(compute_plv(vc_1, vc_2, fs, band)),
        'vc_pli': float(compute_pli(vc_1, vc_2, fs, band)),
        'tc_plv': float(compute_plv(tc_1, tc_2, fs, band)),
        'tc_pli': float(compute_pli(tc_1, tc_2, fs, band)),
        'signals_vc': (vc_1, vc_2),
        'signals_tc': (tc_1, tc_2)
    }

In [ ]:
# Visualization 6 & 7: PLI correctly identifies volume conduction

results = demonstrate_pli_volume_conduction(seed=42)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

fs = 500
band = (8, 12)
nyq = fs / 2
b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')

# Row 1: Volume Conduction
vc_1, vc_2 = results['signals_vc']
vc_1_filt = signal.filtfilt(b, a, vc_1)
vc_2_filt = signal.filtfilt(b, a, vc_2)
phase_diff_vc = np.angle(signal.hilbert(vc_1_filt)) - np.angle(signal.hilbert(vc_2_filt))
phase_diff_vc = np.mod(phase_diff_vc + np.pi, 2*np.pi) - np.pi

axes[0, 0].plot(vc_1[:500], color=COLORS['signal_1'], alpha=0.8, label='Electrode 1')
axes[0, 0].plot(vc_2[:500], color=COLORS['signal_2'], alpha=0.8, label='Electrode 2')
axes[0, 0].set_title('Volume Conduction: Signals', fontweight='bold')
axes[0, 0].legend()

axes[0, 1].hist(phase_diff_vc, bins=50, color=COLORS['warning'], alpha=0.7, density=True)
axes[0, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[0, 1].set_title('Phase Difference Distribution\n(Peaked at 0)', fontweight='bold')
axes[0, 1].set_xlabel('Δφ (rad)')
axes[0, 1].set_xlim(-np.pi, np.pi)

axes[0, 2].bar(['PLV', 'PLI'], [results['vc_plv'], results['vc_pli']], 
               color=[COLORS['signal_1'], COLORS['signal_2']], edgecolor='black')
axes[0, 2].set_ylim(0, 1.1)
axes[0, 2].set_title(f"PLV={results['vc_plv']:.2f} (HIGH!)\nPLI={results['vc_pli']:.2f} (LOW ✓)", 
                     fontweight='bold')

# Row 2: True Connectivity
tc_1, tc_2 = results['signals_tc']
tc_1_filt = signal.filtfilt(b, a, tc_1)
tc_2_filt = signal.filtfilt(b, a, tc_2)
phase_diff_tc = np.angle(signal.hilbert(tc_1_filt)) - np.angle(signal.hilbert(tc_2_filt))
phase_diff_tc = np.mod(phase_diff_tc + np.pi, 2*np.pi) - np.pi

axes[1, 0].plot(tc_1[:500], color=COLORS['signal_1'], alpha=0.8, label='Electrode 1')
axes[1, 0].plot(tc_2[:500], color=COLORS['signal_2'], alpha=0.8, label='Electrode 2')
axes[1, 0].set_title('True Connection: Signals', fontweight='bold')
axes[1, 0].legend()

axes[1, 1].hist(phase_diff_tc, bins=50, color=COLORS['signal_1'], alpha=0.7, density=True)
axes[1, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[1, 1].set_title('Phase Difference Distribution\n(Peaked away from 0)', fontweight='bold')
axes[1, 1].set_xlabel('Δφ (rad)')
axes[1, 1].set_xlim(-np.pi, np.pi)

axes[1, 2].bar(['PLV', 'PLI'], [results['tc_plv'], results['tc_pli']], 
               color=[COLORS['signal_1'], COLORS['signal_2']], edgecolor='black')
axes[1, 2].set_ylim(0, 1.1)
axes[1, 2].set_title(f"PLV={results['tc_plv']:.2f}\nPLI={results['tc_pli']:.2f} (Both HIGH ✓)", 
                     fontweight='bold')

plt.suptitle('PLI Correctly Identifies Volume Conduction\n(PLI is low for VC, high for true connection)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart summary
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(2)
width = 0.35

bars1 = ax.bar(x - width/2, [results['vc_plv'], results['tc_plv']], width, 
               label='PLV', color=COLORS['signal_1'], edgecolor='black')
bars2 = ax.bar(x + width/2, [results['vc_pli'], results['tc_pli']], width, 
               label='PLI', color=COLORS['signal_2'], edgecolor='black')

ax.set_ylabel('Connectivity Value')
ax.set_xticks(x)
ax.set_xticklabels(['Volume Conduction\n(SPURIOUS)', 'True Connection\n(GENUINE)'])
ax.set_ylim(0, 1.1)
ax.legend()
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)

# Add value labels
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points', ha='center', va='bottom')

ax.set_title('PLI Distinguishes What PLV Cannot', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 6: The Sensitivity vs Specificity Trade-off

PLI isn't a free lunch. Let's be honest about its limitations:

**Advantage (Specificity)**: PLI doesn't false-alarm on volume conduction

**Disadvantage (Sensitivity)**: PLI may miss **true zero-lag connections**!
- If true connectivity happens at exactly zero lag, PLI = 0
- Such connections ARE possible (e.g., common input)

**Noise sensitivity**: The sign function is discontinuous
- Small noise fluctuations around 0 cause sign flips
- Can make PLI unstable for small phase differences

**The trade-off**:
- **PLV**: High sensitivity, low specificity (finds all, including false positives)
- **PLI**: High specificity, lower sensitivity (confident results, may miss some)

**Solution**: wPLI (G03) improves on PLI!

In [ ]:
# Visualization 8: Sensitivity vs Specificity diagram

fig, ax = plt.subplots(figsize=(10, 8))

# Draw regions
ax.fill([0.1, 0.9, 0.9, 0.1], [0.1, 0.1, 0.9, 0.9], 
        color='lightgray', alpha=0.3, label='Ideal zone')

# Plot metrics
ax.scatter([0.9], [0.5], s=500, c=COLORS['signal_1'], marker='o', 
           edgecolors='black', linewidths=2, label='PLV', zorder=5)
ax.scatter([0.5], [0.85], s=500, c=COLORS['signal_2'], marker='s', 
           edgecolors='black', linewidths=2, label='PLI', zorder=5)
ax.scatter([0.75], [0.8], s=500, c=COLORS['highlight'], marker='^', 
           edgecolors='black', linewidths=2, label='wPLI (G03)', zorder=5)

# Labels
ax.set_xlabel('Sensitivity\n(Detecting true connections)', fontsize=12)
ax.set_ylabel('Specificity\n(Rejecting volume conduction)', fontsize=12)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

# Annotations
ax.annotate('PLV: Finds all\nincluding false +', xy=(0.9, 0.5), xytext=(0.65, 0.3),
            fontsize=10, ha='center',
            arrowprops=dict(arrowstyle='->', color='gray'))
ax.annotate('PLI: Confident but\nmay miss some', xy=(0.5, 0.85), xytext=(0.25, 0.7),
            fontsize=10, ha='center',
            arrowprops=dict(arrowstyle='->', color='gray'))

ax.legend(loc='lower right')
ax.set_title('PLV vs PLI: The Sensitivity-Specificity Trade-off\n(No free lunch!)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Section 7: PLI Matrix

Just like PLV, we can compute PLI for all channel pairs.

In [ ]:
# Function 5: compute_pli_matrix

def compute_pli_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """
    Compute PLI matrix for all channel pairs.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Multi-channel data, shape (n_channels, n_samples).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    NDArray[np.float64]
        PLI matrix, shape (n_channels, n_channels).
        Diagonal is 0 (not 1 like PLV, since self-PLI is undefined).
    """
    n_channels = data.shape[0]
    
    # Filter all channels
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    data_filt = signal.filtfilt(b, a, data, axis=1)
    
    # Extract phases for all channels
    phases = np.angle(signal.hilbert(data_filt, axis=1))
    
    # Compute PLI matrix
    pli_matrix = np.zeros((n_channels, n_channels))
    
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            pli = compute_pli_from_phases(phases[i], phases[j])
            pli_matrix[i, j] = pli
            pli_matrix[j, i] = pli  # Symmetric
    
    return pli_matrix

In [ ]:
# Function 6: compute_pli_matrix_bands

STANDARD_BANDS = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

def compute_pli_matrix_bands(
    data: NDArray[np.float64],
    fs: float,
    bands: dict[str, tuple[float, float]] | None = None
) -> dict[str, NDArray[np.float64]]:
    """
    Compute PLI matrices for multiple frequency bands.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Multi-channel data, shape (n_channels, n_samples).
    fs : float
        Sampling frequency in Hz.
    bands : dict[str, tuple[float, float]] | None, optional
        Dictionary of band names to (low, high) frequencies.
        If None, uses standard bands.
    
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary of band names to PLI matrices.
    """
    if bands is None:
        bands = STANDARD_BANDS
    
    return {name: compute_pli_matrix(data, fs, band) for name, band in bands.items()}

In [ ]:
# Visualization 9: PLV vs PLI matrices

# Generate synthetic data with mix of volume conduction and true connections
np.random.seed(42)
fs = 500
n_samples = 10000
n_channels = 6
t = np.arange(n_samples) / fs
freq = 10

data = np.zeros((n_channels, n_samples))

# Source signals
source_1 = np.sin(2 * np.pi * freq * t)  # Shared source (volume conduction)
source_2 = np.sin(2 * np.pi * freq * t + np.pi/3)  # Delayed source (true connection)

# Channels 0-2: Share source_1 (volume conduction)
for i in range(3):
    data[i] = source_1 + 0.2 * np.random.randn(n_samples)

# Channels 3-5: True connectivity with delays
delays = [0, 10, 20]  # Different delays in samples
for i, delay in enumerate(delays):
    data[i + 3] = np.roll(source_2, delay) + 0.3 * np.random.randn(n_samples)

# Compute PLV matrix (for comparison)
def compute_plv_matrix(data, fs, band):
    n_channels = data.shape[0]
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    data_filt = signal.filtfilt(b, a, data, axis=1)
    phases = np.angle(signal.hilbert(data_filt, axis=1))
    
    plv_matrix = np.ones((n_channels, n_channels))
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            phase_diff = phases[i] - phases[j]
            plv = np.abs(np.mean(np.exp(1j * phase_diff)))
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv
    return plv_matrix

band = (8, 12)
plv_matrix = compute_plv_matrix(data, fs, band)
pli_matrix = compute_pli_matrix(data, fs, band)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PLV
im1 = axes[0].imshow(plv_matrix, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(im1, ax=axes[0], label='PLV')
axes[0].set_title('PLV Matrix\n(Volume conduction pairs show HIGH)', fontweight='bold')
axes[0].set_xticks(range(n_channels))
axes[0].set_yticks(range(n_channels))
axes[0].set_xticklabels(['VC1', 'VC2', 'VC3', 'True1', 'True2', 'True3'])
axes[0].set_yticklabels(['VC1', 'VC2', 'VC3', 'True1', 'True2', 'True3'])

# Draw block boundaries
axes[0].axhline(y=2.5, color='white', linewidth=2)
axes[0].axvline(x=2.5, color='white', linewidth=2)

# PLI
im2 = axes[1].imshow(pli_matrix, cmap='viridis', vmin=0, vmax=1)
plt.colorbar(im2, ax=axes[1], label='PLI')
axes[1].set_title('PLI Matrix\n(Volume conduction pairs show LOW)', fontweight='bold')
axes[1].set_xticks(range(n_channels))
axes[1].set_yticks(range(n_channels))
axes[1].set_xticklabels(['VC1', 'VC2', 'VC3', 'True1', 'True2', 'True3'])
axes[1].set_yticklabels(['VC1', 'VC2', 'VC3', 'True1', 'True2', 'True3'])

# Draw block boundaries
axes[1].axhline(y=2.5, color='white', linewidth=2)
axes[1].axvline(x=2.5, color='white', linewidth=2)

plt.suptitle('PLV vs PLI Matrices: Volume Conduction Effects Visible', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 8: PLI for Hyperscanning

In [ ]:
# Function 7: compute_pli_hyperscanning

def compute_pli_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, NDArray[np.float64]]:
    """
    Compute PLI matrices for hyperscanning data.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Data from participant 1, shape (n_channels_p1, n_samples).
    data_p2 : NDArray[np.float64]
        Data from participant 2, shape (n_channels_p2, n_samples).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary with 'within_p1', 'within_p2', 'between', and 'full' matrices.
    """
    n_ch_p1 = data_p1.shape[0]
    n_ch_p2 = data_p2.shape[0]
    
    # Filter and extract phases
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    
    data_p1_filt = signal.filtfilt(b, a, data_p1, axis=1)
    data_p2_filt = signal.filtfilt(b, a, data_p2, axis=1)
    
    phases_p1 = np.angle(signal.hilbert(data_p1_filt, axis=1))
    phases_p2 = np.angle(signal.hilbert(data_p2_filt, axis=1))
    
    # Within-P1 matrix
    within_p1 = np.zeros((n_ch_p1, n_ch_p1))
    for i in range(n_ch_p1):
        for j in range(i + 1, n_ch_p1):
            pli = compute_pli_from_phases(phases_p1[i], phases_p1[j])
            within_p1[i, j] = pli
            within_p1[j, i] = pli
    
    # Within-P2 matrix
    within_p2 = np.zeros((n_ch_p2, n_ch_p2))
    for i in range(n_ch_p2):
        for j in range(i + 1, n_ch_p2):
            pli = compute_pli_from_phases(phases_p2[i], phases_p2[j])
            within_p2[i, j] = pli
            within_p2[j, i] = pli
    
    # Between matrix
    between = np.zeros((n_ch_p1, n_ch_p2))
    for i in range(n_ch_p1):
        for j in range(n_ch_p2):
            between[i, j] = compute_pli_from_phases(phases_p1[i], phases_p2[j])
    
    # Full matrix
    n_total = n_ch_p1 + n_ch_p2
    full = np.zeros((n_total, n_total))
    full[:n_ch_p1, :n_ch_p1] = within_p1
    full[n_ch_p1:, n_ch_p1:] = within_p2
    full[:n_ch_p1, n_ch_p1:] = between
    full[n_ch_p1:, :n_ch_p1] = between.T
    
    return {
        'within_p1': within_p1,
        'within_p2': within_p2,
        'between': between,
        'full': full
    }

---

## Section 9: Signed PLI (Optional)

Standard PLI uses absolute value and loses direction. **Signed PLI** preserves it:

- Range: -1 to +1
- Positive: Y leads X on average
- Negative: X leads Y on average

**Caution**: Sign depends on channel order!

In [ ]:
# Function 8: compute_signed_pli

def compute_signed_pli(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute Signed PLI (without absolute value).
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of first signal (radians).
    phase_y : NDArray[np.float64]
        Instantaneous phase of second signal (radians).
    
    Returns
    -------
    float
        Signed PLI between -1 and +1.
        Positive = Y leads X, Negative = X leads Y.
    """
    phase_diff = phase_x - phase_y
    signed_pli = np.mean(np.sign(np.sin(phase_diff)))
    return float(signed_pli)

---

## Section 10: Hands-On Exercises

In [ ]:
# Exercise 1: PLI Basics
# Generate two signals with consistent 45° phase lag
# Compute PLV and PLI - both should be high (true connectivity)

# YOUR CODE HERE

In [ ]:
# Exercise 2: Volume Conduction Test
# Simulate volume conduction (same signal + small noise at two electrodes)
# Compute PLV → should be high
# Compute PLI → should be low!

# YOUR CODE HERE

In [ ]:
# Exercise 3: Sign Distribution
# Generate phase-locked signals
# Compute phase differences
# Histogram the signs (+1, -1)
# For true connectivity: imbalanced
# For volume conduction: balanced

# YOUR CODE HERE

In [ ]:
# Exercise 4: Zero-Lag True Connection
# Create two signals with genuine zero-lag relationship (common input)
# Compute PLI → should be low (even though connection is "real")
# This demonstrates PLI's limitation

# YOUR CODE HERE

---

## Summary

### Key Takeaways

1. **PLV problem**: High for volume conduction (zero-lag)

2. **PLI solution**: Measures asymmetry of phase difference SIGN
   - Formula: PLI = |mean(sign(sin(Δφ)))|
   - Range: 0 (symmetric/volume conduction) to 1 (fully asymmetric)

3. **How it works**:
   - Volume conduction → Δφ ≈ 0 → signs balance → PLI ≈ 0
   - True connectivity → consistent lag → signs imbalanced → PLI > 0

4. **Trade-off**: Higher specificity but lower sensitivity than PLV
   - May miss true zero-lag connections
   - Sign function makes PLI sensitive to noise

5. **For hyperscanning**: PLI is safer for within-brain; between-brain less critical

6. **Signed PLI**: Preserves lead/lag direction

7. **Next**: wPLI (G03) improves noise robustness

---

## Discussion Questions

1. You find PLV = 0.7 but PLI = 0.1 between two nearby electrodes. What's your interpretation?

2. PLI = 0 could mean volume conduction OR true zero-lag coupling. How problematic is this ambiguity?

3. The sign function in PLI is discontinuous. How might this affect PLI stability? (Hint: G03)

4. For between-brain hyperscanning, is PLI necessary since there's no volume conduction?

5. You want to compare connectivity between patients and controls. Would you use PLV or PLI?